In [1]:
from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.agent.self_refine import SelfRefineAgent
from agential.cog.prompts.self_refine import (
    SELF_REFINE_CRITIQUE_INSTRUCTION_GSM8K,
    SELF_REFINE_INSTRUCTION_GSM8K,
    SELF_REFINE_REFINE_INSTRUCTION_GSM8K,
    GSM8K_CRITIQUE_FEWSHOT_EXAMPLES,
    GSM8K_REFINE_FEWSHOT_EXAMPLES
)
from agential.cog.prompts.benchmarks.gsm8k import GSM8K_FEWSHOT_EXAMPLES_POT

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

llm = ChatOpenAI()

In [ ]:
agent = SelfRefineAgent(llm)

In [ ]:
question = "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?"

In [ ]:
agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=SELF_REFINE_INSTRUCTION_GSM8K,
    feedback_examples=GSM8K_CRITIQUE_FEWSHOT_EXAMPLES,
    feedback_prompt=SELF_REFINE_CRITIQUE_INSTRUCTION_GSM8K,
    refine_examples=GSM8K_REFINE_FEWSHOT_EXAMPLES,
    refine_prompt=SELF_REFINE_REFINE_INSTRUCTION_GSM8K
)